In [1]:
!python -m spacy download en_core_web_sm
import spacy

nlp = spacy.load('en_core_web_sm')

/Users/jeremyspradlin/.local/share/virtualenvs/Lambda-School-Labs-human-rights-first-asyl-Kg_LyPhr/bin/python: No module named spacy


ModuleNotFoundError: No module named 'spacy'

In [ ]:
!pip3 install Pillow
!pip3 install pytesseract
!pip3 install pdf2image
!sudo apt-get install tesseract-ocr

Password:

In [3]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [41.5 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ge

In [4]:
!apt-get install poppler-utils --fix-missing

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 27 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.12 [154 kB]
Fetched 154 kB in 2s (102 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 145527 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.12_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.12) ...
Setting up poppler-utils (0.62.0-2ubuntu2.12) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [6]:
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os 
import re
import spacy

In [7]:
pdfs = [file for file in os.listdir() if '.pdf' in file]
pdfs

['457374430-A-E-M-P-AXXX-XXX-795-BIA-March-16-2020.pdf',
 '473847009-A-J-V-U-AXXX-XXX-645-BIA-July-10-2020.pdf',
 '477308213-M-S-F-AXXX-XXX-188-BIA-Aug-27-2020.pdf',
 '386774234-D-T-AXXX-XXX-959-BIA-July-27-2018.pdf',
 '452083522-M-A-D-A-AXXX-XXX-603-BIA-Feb-11-2020.pdf',
 '470814353-K-I-AXXX-XXX-247-BIA-July-7-2020.pdf',
 '431711050-C-N-A-AXXX-XXX-484-BIA-Sept-12-2019.pdf',
 '427098696-Y-M-G-L-AXXX-XXX-119-BIA-Sept-4-2019.pdf',
 '447696312-J-V-S-E-AXXX-XXX-159-BIA-Jan-10-2020.pdf',
 '462824675-D-E-D-L-AXXX-XXX-662-BIA-April-10-2020.pdf',
 '431716865-N-G-M-AXXX-XXX-926-BIA-Oct-4-2019.pdf',
 '470813340-J-S-R-Z-AXXX-XXX-188-BIA-June-24-2020.pdf',
 '431716768-M-H-AXXX-XXX-860-BIA-Oct-4-2019.pdf',
 '473846393-K-I-AXXX-XXX-247-BIA-July-7-2020.pdf',
 '470810825-M-M-A-AXXX-XXX-675-BIA-June-9-2020.pdf',
 '462828817-Z-M-AXXX-XXX-820-BIA-April-30-2020.pdf']

In [8]:
os.path.isdir('./temp/')

False

In [9]:
def ocr_func(pdf_file, txt_folder = './temp/'):
    '''
    Takes an uploaded .pdf file, converts it to plain text, and saves it as a
    .txt file
    '''
    #update to recieve input .pdf file as a variable?

    pages = convert_from_path(pdf_file, dpi=300)

    num_pages = 0

    for image_counter, page in enumerate(pages):
        filename = 'page_' + str(image_counter) + '.jpg'
        page.save(filename, 'JPEG')
        num_pages += 1
    if not os.path.isdir(txt_folder):
      os.mkdir(txt_folder)
    outfile = ''.join(['./temp/',pdf_file.split('.')[0], '.txt'])
    fulltext = []
    f = open(outfile, 'a')

    for i in range(num_pages):
        filename = 'page_' + str(i) + '.jpg'
        text = str(((pytesseract.image_to_string(Image.open(filename)))))
        os.remove(filename)
        # os.remove(pdf_file)
        text = text.replace('-\n', '')
        fulltext.append(text)
        f.write(text)

    f.close()

    return (outfile[len(txt_folder):],''.join(fulltext).split('\n\n'))

In [11]:
def getJudge(text: list) -> tuple:
        '''
        In an appeal document, finds the name of the judge on the appeal
        returns a tuple of 'high confidence' or 'low confidence' 
        and the name of the judge
        If the word 'judge' is near the token, high confidence is returned
        This is because the judge name is followed or preceded by their title
        '''
        nlp = spacy.load('en_core_web_sm')
        spaceSepText = ' '.join(text)
        doc = nlp(spaceSepText)
        people = [ent for ent in doc.ents if ent.label_ == 'PERSON'
                        and len(ent.text.split(' ')) in (2,3)][-5:]
        possibleJudges = [doc[person.start-4:person.end+4] for person in people]
        # print(possibleJudges)
        for idx, tokens in enumerate(possibleJudges):
            if 'judge' in str(tokens).lower():
              return ('High confidence', people[idx])
        return ('low confidence', max(sorted(people, key = len)))

def isAppeal(text):
  appeal = re.compile(r"(IN REMOVAL PROCEEDINGS\nAPPEAL($|\n)|\nAPPEAL\n)")
  return appeal.search('\n'.join(text))

In [12]:
def getCourt(text: list):
  nlp = spacy.load('en_core_web_sm')
  spaceSepText = ' '.join(text)
  doc = nlp(spaceSepText)
  court = [ent for ent in doc.ents if ent.label_ == 'GPE']
  return court[0].text + ', ' +  court[1].text

def getCoO(text: list):
  nlp = spacy.load('en_core_web_sm')
  spaceSepText = ' '.join(text)
  doc = nlp(spaceSepText)
  places = [ent for ent in doc.ents if ent.label_ == 'GPE']
  possibleCoO = [doc[place.start-4:place.end+4] for place in places]
  for idx, token in enumerate(possibleCoO):
    return max(sorted(places, key = len))


In [13]:
appeals = []
originals = []
print(len(fullTextList))
for fullText in fullTextList:
  print('\nfull text', fullText[0])
  if isAppeal(fullText[1]):
    print('Appeal Case')
    judge = getJudge(fullText[1])
    print('print judge', judge)
    court = getCourt(fullText[1])
    print('print court', court)
    coo = getCoO(fullText[1])
    print('print country of origin', coo)
    if fullText not in appeals:
      appeals.append(fullText)
  else:
    print('Original Case')
    judge = getJudge(fullText[1])
    print('print judge: ', judge)
    court = getCourt(fullText[1])
    print('print court: ', court)
    coo = getCoO(fullText[1])
    print('print country of origin: ', coo)
    if fullText not in originals:
      originals.append(fullText)
    
    # for x in fullText[1]:
    #   print(x)


16

full text 457374430-A-E-M-P-AXXX-XXX-795-BIA-March-16-2020.txt
Appeal Case
print judge ('High confidence', de novo)
print court Falls Church, Virginia
print country of origin El Salvador

full text 473847009-A-J-V-U-AXXX-XXX-645-BIA-July-10-2020.txt
Appeal Case
print judge ('low confidence', AXXX XXX 645)
print court Falls Church, Virginia
print country of origin Maracaibo

full text 477308213-M-S-F-AXXX-XXX-188-BIA-Aug-27-2020.txt
Appeal Case
print judge ('low confidence', AXXX XXX)
print court Falls Church, Virginia
print country of origin Cuba

full text 386774234-D-T-AXXX-XXX-959-BIA-July-27-2018.txt
Appeal Case
print judge ('low confidence', AXXX XXX)
print court Falls Church, Virginia
print country of origin Cameroon

full text 452083522-M-A-D-A-AXXX-XXX-603-BIA-Feb-11-2020.txt
Appeal Case
print judge ('low confidence', AXXX XXX)
print court Falls Church, Virginia
print country of origin El Salvador

full text 470814353-K-I-AXXX-XXX-247-BIA-July-7-2020.txt
Appeal Case
print j

In [10]:
fullTextList = [ocr_func(PDF_file) for PDF_file in pdfs]
for file, text in fullTextList:
  print(file)
  print(text[:10])

457374430-A-E-M-P-AXXX-XXX-795-BIA-March-16-2020.txt
['U.S. Department of Justice\nExecutive Office for Immigration Review', 'Board of Immigration Appeals\nOffice of the Clerk', ' ', '5107 Leesburg Pike. Suite 2000\nFalls Church, Virginia 22041', 'Martinez, Alejandra DHS/ICE Office of Chief Counsel - PSD\nDe Mott, McChesney, Curtright, 566 Veterans Drive\n& Armendariz, LLP Pearsall, TX 78061', '8023 Vantage Dr., Ste 800\nSan Antonio, TX 78230', 'Name: VS-PaS, AME... A -795\nDate of this notice: 3/16/2020', "Enclosed is a copy of the Board's decision and order in the above-referenced case.", 'Sincerely,', 'Donna Carr\nChief Clerk']
473847009-A-J-V-U-AXXX-XXX-645-BIA-July-10-2020.txt
['U.S. Department of Justice\nExecutive Office for Immigration Review', 'Board of Immigration Appeals\nOffice of the Clerk', ' ', '5107 Leesburg Pike, Suite 2000\nFalls Church, Virginia 22041', 'Goodwin, Jodilyn Marie DHS/ICE Office of Chief Counsel - HLG\nLaw Office of Jodi Goodwin 1717 Zoy Street\n1221 Eas

In [ ]:
for name, t in appeals or original:
  print(name)

Bain_Asylum Grant Gender-Based Imputed PO_EL SALVADOR_July 2019.txt


In [ ]:
# import shutil

# #using make_archive method to zip a whole directory
# shutil.make_archive('texts', 'zip', './temp/')

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
docs = []
for text in appeals or original:
  docs.append([ent for ent in nlp(' '.join(text[1])).ents])# if ent.label_ == 'PERSON'])

In [ ]:
fullpeopleLists = []
for doc in docs:
  people = []
  for ent in doc:
    if ent.label_ == 'PERSON':
      people.append(ent.text)
  fullpeopleLists.append(people)

In [ ]:
for idx, peopleList in enumerate(fullpeopleLists):
  print(fullTextList[idx][0][7:])
  if any([len(person.split(' ')) >= 3 for person in peopleList[-3:]]):
    print(peopleList[-3:])
    last = peopleList[-3:]
    print(max(last, key = len) )
  else:
    print('not found')
    print(peopleList[-3:])

ylum Grant Gender-Based Imputed PO_EL SALVADOR_July 2019.txt
['the Lead Respondent', 'Zuh', 'Quynh V. Bain']
the Lead Respondent


In [ ]:
placeList = []
for doc in docs:
  places = []
  for ent in doc:
    if ent.label_ == 'GPE':
      places.append(ent.text)
  placeList.append(places)

In [ ]:
#getCourt(original[0][1])
getCoO(appeals[0][1])

the United States Court of Appeals